<a href="https://colab.research.google.com/github/ErnstHolger/Machine-Learning-For-Manufacturing/blob/main/MasterClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from IPython.display import Image,display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from time import sleep
! mkdir /content/images &> /dev/null
! cp --force -r  /content/drive/MyDrive/Master_Class_Slides/ /content/images/
! chmod ogu+rwx /content/images/Master_Class_Slides/*.*

In [ ]:
def show_slides(i):
  return Image(filename='/content/images/Master_Class_Slides/Slide'+str(i)+'.PNG')

In [ ]:
display(interact(show_slides, i=widgets.IntSlider(min=1, max=11, step=1, value=1, layout=widgets.Layout(width='50%'))))

interactive(children=(IntSlider(value=1, description='i', layout=Layout(width='50%'), max=11, min=1), Output()…

<function __main__.show_slides>

Ernst Holger Amort https://www.linkedin.com/in/holgeramort/<br>
TQS Integration https://www.tqsintegration.com/

<h1>Advanced Analytics in Manufacturing</h1>

Data Set: http://www.industrialpenicillinsimulation.com/

<i>Industrial-scale penicillin simulation (IndPenSim) is a first principles mathematical model of a Penicillium chrysogenum fermentation.</i>

<li>Batches 1-30: Controlled by recipe driven approach</li>
<li>Batches 31-60: Controlled by operators</li>
<li>Batches 61:90: Controlled by an Advanced Process Control (APC) solution using the Raman spectroscopy</li>
<li><b>Batches 91:100:</b> Contain faults resulting in process deviations.</li>



In [ ]:
from google.colab import data_table
from ipywidgets import interact, interactive, fixed, interact_manual
from os.path import exists

import pandas as pd
import numpy as np
from plotly.graph_objs import Layout
import plotly.express as px
import plotly.graph_objects as go
import psycopg2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data_table.enable_dataframe_formatter()

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    font_family="Courier New",
    font_size=12,
    font_color="black",
    title_font_family="Courier New",
    title_font_color="black",
    title_font_size=12,
)


In [ ]:
# load data set
if not exists('Mendeley_data/100_Batches_IndPenSim_V3.pkl'):
  !mkdir /content/data &> /dev/null
  !cp --force '/content/drive/MyDrive/Data/100_Batches_IndPenSim.zip' '/content/data/'
  !unzip -o -qq '/content/data/100_Batches_IndPenSim.zip'
  df=pd.read_csv('Mendeley_data/100_Batches_IndPenSim_Statistics.csv')
  df.to_pickle('Mendeley_data/100_Batches_IndPenSim_Statistics.pkl')
  df=pd.read_csv('Mendeley_data/100_Batches_IndPenSim_V3.csv')
  df.to_pickle('Mendeley_data/100_Batches_IndPenSim_V3.pkl')
  #!cp 'Mendeley_data/100_Batches_IndPenSim_V3.pkl' '/content/drive/MyDrive/Data'
# production overview
df=pd.read_pickle('Mendeley_data/100_Batches_IndPenSim_Statistics.pkl')
df

,Batch ref,Penicllin_harvested_during_batch(kg),Penicllin_harvested_end_of_batch (kg),Penicllin_yield_total (kg),Fault ref(0-NoFault 1-Fault)
0,1,1066400.0,1720000.0,2786400.0,0
1,2,985910.0,1340100.0,2326000.0,0
2,3,1416100.0,1259200.0,2675300.0,0
3,4,815700.0,1071000.0,1886700.0,0
4,5,1128500.0,2434400.0,3562900.0,0
5,6,1005700.0,1383100.0,2388700.0,0
6,7,1147600.0,2307000.0,3454600.0,0
7,8,1201200.0,2676200.0,3877400.0,0
8,9,1757800.0,2325300.0,4083100.0,0
9,10,878430.0,1103600.0,1982000.0,0


<h2>Create a physical data model</h2>
1.   Translate process variables into Human Readable Names<br>
2.   Use consistent naming convention such as Temperature, Pressure, ...<br>
3.   Separate attribute names and units<br>
4.   This step is mostly manual!

<i> In OSIsoft the physical model is called Asset Framework and the recipe model Event Framework.</i>

In [ ]:
# Variable Mapping
mapping={ 'Batch ref': 'Batch Reference',
        'Penicllin_harvested_during_batch(kg)': 'Penicillin',
        'Penicllin_harvested_end_of_batch (kg)': 'Penicillin Final',
        'Penicllin_yield_total (kg)': 'Penicillin Yield',
        'Time (h)': 'Time',
        'Aeration rate(Fg:L/h)': 'Aeration',
        'Agitator RPM(RPM:RPM)': 'Agitator',
        'Sugar feed rate(Fs:L/h)': 'Sugar Feed',
        'Acid flow rate(Fa:L/h)': 'Acid Flow',
        'Base flow rate(Fb:L/h)': 'Base Flow',
        'carbon dioxide percent in off-gas(CO2outgas:%)':'CO2 Off-Gas',
        'Heating/cooling water flow rate(Fc:L/h)': 'Heating Cooling Flow',
        'Heating water flow rate(Fh:L/h)': 'Heating Water Flow',
        'Water for injection/dilution(Fw:L/h)': 'Water Injection',
        'Air head pressure(pressure:bar)': 'Air Head Pressure',
        'Dumped broth flow(Fremoved:L/h)': 'Dumped Broth Flow',
        'Substrate concentration(S:g/L)': 'Substrate Concentration',
        'Dissolved oxygen concentration(DO2:mg/L)': 'Dissolved Oxygen',
        'Penicillin concentration(P:g/L)': 'Penicillin Concentration',
        'Vessel Volume(V:L)': 'Vessel Volume',
        'Vessel Weight(Wt:Kg)': 'Vessel Weight',
        'pH(pH:pH)': 'pH',
        'Temperature(T:K)': 'Temperature',
        'Generated heat(Q:kJ)': 'Generated Heat',
        'CO2 Off-Gas': 'CO2',
        'PAA flow(Fpaa:PAA flow (L/h))': 'PAA Flow',
        'PAA concentration offline(PAA_offline:PAA (g L^{-1}))': 'PAA Concentration',
        'Oil flow(Foil:L/hr)': 'Oil Flow',
        'NH_3 concentration off-line(NH3_offline:NH3 (g L^{-1}))': 'NH_3 Concentration',
        'Oxygen Uptake Rate(OUR:(g min^{-1}))': 'Oxygen Uptake Rate',
        'Oxygen in percent in off-gas(O2:O2  (%))': 'Oxygen Percent',
        'Offline Penicillin concentration(P_offline:P(g L^{-1}))': 'Penicillin Concentration Offline',
        'Offline Biomass concentratio(X_offline:X(g L^{-1}))':'Biomass concentration Offline',
        'Carbon evolution rate(CER:g/h)':'Carbon evolution',
        'Ammonia shots(NH3_shots:kgs)':'Ammonia Shots',
        'Viscosity(Viscosity_offline:centPoise)': 'Viscosity',
        'Fault reference(Fault_ref:Fault ref)': 'Fault Reference',
        '0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)': 'Recipe Driven',
        '1- No Raman spec': 'No Raman',
        ' 1-Raman spec recorded': 'Raman',
        '2-PAT control(PAT_ref:PAT ref)':'Batch Id',
        'Batch reference(Batch_ref:Batch ref)':'PAT Control',
        'Batch ID':'Id',
        'Fault flag':'Fault Flag'
    }
independent={
    'Id':'Batch Id','Fault Flag':'Fault Flag',
    'Ti':'Time','Fg':'Aeration',
    'RPM':'Agitator','Fs':'Sugar Feed',
    'Fa':'Acid Flow','Fb':'Base Flow',
    'Fc':'Heating Cooling Flow','Fh':'Heating Water Flow',
    'Fw':'Water Injection','pressure':'Air Head Pressure',
    'Fremoved':'Dumped Broth Flow','DO2':'Dissolved Oxygen',
    'V':'Vessel Volume','Wt':'Vessel Weight',
    'pH':'pH','T':'Temperature','Q':'Generated Heat',
    'CO2outgas':'CO2 Off-Gas','Fpaa':'PAA Flow',
    'Foil':'Oil Flow','OUR':'Oxygen Uptake Rate',
    'O2':'Oxygen Percent','CER':'Carbon evolution' 
}

In [ ]:
# splitting into process variables (df) and online measurements
if not exists('Mendeley_data/100_Batches_IndPenSim_V3_Raman.pkl') or \
  not exists('Mendeley_data/100_Batches_IndPenSim_V3_Independent.pkl'):
  df=pd.read_pickle('Mendeley_data/100_Batches_IndPenSim_V3.pkl')
  df=df.rename(columns={'2-PAT control(PAT_ref:PAT ref)':'Batch Id','Time (h)': 'Time'})
  n=len(df.columns)
  idx=[0,38]+list(range(40,n))
  raman=df.iloc[idx]
  raman.to_pickle('Mendeley_data/100_Batches_IndPenSim_V3_Raman.pkl')
  !cp 'Mendeley_data/100_Batches_IndPenSim_V3_Raman.pkl' '/content/drive/MyDrive/Data'
  df=df.iloc[:,:39]
  df.to_pickle('Mendeley_data/100_Batches_IndPenSim_V3_Independent.pkl')
  !cp 'Mendeley_data/100_Batches_IndPenSim_V3_Independent.pkl' '/content/drive/MyDrive/Data'

In [ ]:
# plot definitions
lc={
    'royalblue':{'color':'royalblue', 'width':1,},
    'firebrick':{'color':'firebrick', 'width':1,},
    'black':{'color':'black', 'width':1,},
    'lightgray-3':{'color':'lightgray', 'width':3,}
}
def add_line(fig,x,y,name,line):
  fig.add_trace(go.Scatter(x=x, y=y,mode='lines', 
                           name=name,line=line))
def plot_raw(variable, df):
  tbl = df.pivot(index='Time',columns='Batch Id',\
                   values=variable).reset_index().rename_axis(None, axis=1)
  fig = go.Figure()
  for column in tbl.columns:
    if column=='Time': continue
    n=int(column)
    if n<=90: line = {'color':'lightgray', 'width':3,}
    else: line = {'color':'firebrick', 'width':1,}
    add_line(fig,tbl['Time'], tbl[column],column,line)
  fig.update_layout(layout)
  fig.update_layout(title="Batch Overlay",xaxis_title="Time [hours]",
    yaxis_title=variable, legend_title="Batch Id")
  fig.show()

def plot_limit(df,limit,variable,batchid):
  _sample=df[df['Batch Id']==batchid][['Time',variable]]
  _limit=limit[limit['name']==variable]
  fig = go.Figure()
  add_line(fig,_limit.index, _limit['lower'],'lower',lc['royalblue'])
  add_line(fig,_limit.index, _limit['upper'],'upper',lc['firebrick'])
  add_line(fig,_sample['Time'], _sample[variable],str(batchid),lc['black'])
  fig.update_layout(layout)
  fig.update_layout(
    title="Univariate Control Limits",
    xaxis_title="Time [hours]",
    yaxis_title=variable,
    legend_title="Control Limit")
  fig.show()

def plot_limit_animation(df,limit,variable,batchid):
  _sample=df[df['Batch Id']==batchid][['Time',variable]]
  _limit=limit[limit['name']==variable]
  fig = go.Figure()
  add_line(fig,_limit.index, _limit['lower'],'lower',lc['royalblue'])
  add_line(fig,_limit.index, _limit['upper'],'upper',lc['firebrick'])
  add_line(fig,_sample['Time'], _sample[variable],str(batchid),lc['black'])
  fig.update_layout(layout)
  fig.update_layout(
    title="Univariate Control Limits",
    xaxis_title="Time [hours]",
    yaxis_title=variable,
    legend_title="Control Limit")
  fig.show()



In [ ]:
display(interact(show_slides, i=widgets.IntSlider(min=1, max=11, step=1, value=6, layout=widgets.Layout(width='50%'))))

interactive(children=(IntSlider(value=6, description='i', layout=Layout(width='50%'), max=11, min=1), Output()…

<function __main__.show_slides>

In [ ]:
df=pd.read_pickle('Mendeley_data/100_Batches_IndPenSim_V3_Independent.pkl')
df=df.rename(columns=mapping)
interact(plot_raw, variable=list(df.columns),df=fixed(df));

interactive(children=(Dropdown(description='variable', options=('Time', 'Aeration', 'Agitator', 'Sugar Feed', …

In [ ]:
def calc_limit(df):
  res=None
  for column in df.columns:
    if column=='Time': continue
    tbl = df.groupby('Time').agg({column: ['mean','min','max','std']})
    tbl.columns=tbl.columns.get_level_values(1)
    tbl['name']=column
    tbl['std']=tbl['std'].fillna(0)
    tbl['lower']=tbl['mean']-3*tbl['std']
    tbl['upper']=tbl['mean']+3*tbl['std']
    tbl['lower']=tbl[["lower", "min"]].max(axis=1) # lower >= min
    tbl['upper']=tbl[["upper", "max"]].min(axis=1) # upper <= max
    tbl=tbl[['name','lower','mean','upper']]
    try:
      res = pd.concat([res,tbl])
    except:
      res=tbl
  return res  

In [ ]:
df=pd.read_pickle('Mendeley_data/100_Batches_IndPenSim_V3_Independent.pkl')
df=df.rename(columns=mapping)
id=list(range(1,90,1))
train=df[df['Batch Id'].isin(id)]
limit=calc_limit(train)
interact(plot_limit,
    df=fixed(df),
    limit=fixed(limit),
    variable=list(df.columns),
    batchid=widgets.IntSlider(min=1, max=100, step=1, value=10,layout=widgets.Layout(width='50%')));

interactive(children=(Dropdown(description='variable', options=('Time', 'Aeration', 'Agitator', 'Sugar Feed', …

In [ ]:
display(interact(show_slides, i=widgets.IntSlider(min=1, max=9, step=1, value=8, layout=widgets.Layout(width='50%'))))

interactive(children=(IntSlider(value=8, description='i', layout=Layout(width='50%'), max=9, min=1), Output())…

<function __main__.show_slides>

In [ ]:
def prep_df(df,idx):
  df=df[df['Batch Id'].isin(idx)]
  df=df.rename(columns=mapping)
  df=df[independent.values()]
  df.sort_values(by=['Time','Batch Id'],inplace=True)
  y=df['Time']
  id=df['Batch Id']
  df.drop(['Batch Id', 'Fault Flag', 'Time'], axis=1, inplace=True)
  return df, y, id

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn import preprocessing

train=list(range(1,91))
test=list(range(91,101))

df=pd.read_pickle('Mendeley_data/100_Batches_IndPenSim_V3_Independent.pkl')
x,y, id=prep_df(df,train)
xt,yt,id=prep_df(df,test)

x_scaler = preprocessing.StandardScaler().fit(x)
y_scaler = preprocessing.StandardScaler().fit(y.values.reshape(-1, 1))

xs=x_scaler.transform(x)
ys=y_scaler.transform(y.values.reshape(-1, 1))
xt=x_scaler.transform(xt)


pls = PLSRegression(n_components=1)
pls.fit(xs, ys)

T = pls.x_scores_
W = pls.x_weights_
P = pls.x_loadings_
R = pls.x_rotations_

trdf=pd.DataFrame()
trdf['Time']=y.values
trdf['T1']=T.squeeze()
limit=calc_limit(trdf)

tedf=pd.DataFrame()
tedf['Time']=yt.values
tedf['T1']=(xt@P).squeeze()

fig = go.Figure()
add_line(fig,limit.index, limit['lower'],'lower',lc['royalblue'])
add_line(fig,limit.index, limit['upper'],'upper',lc['firebrick'])
# add_line(fig,_sample['Time'], _sample[variable],str(batchid),lc['black'])
fig.update_layout(layout)
fig.update_layout(
  title="Multivariate Control Limits",
  xaxis_title="Time [hours]",
  yaxis_title="t-Score",
  legend_title="Control Limit")
fig.show()

In [ ]:
(xt@P).shape


(11525, 1)

In [ ]:
display(interact(show_slides, i=widgets.IntSlider(min=1, max=11, step=1, value=9, layout=widgets.Layout(width='50%'))))

interactive(children=(IntSlider(value=9, description='i', layout=Layout(width='50%'), max=11, min=1), Output()…

<function __main__.show_slides>

In [ ]:
import plotly.express as px
df = px.data.gapminder()
print(df.head())
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", 
           hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

       country continent  year  lifeExp       pop   gdpPercap iso_alpha  \
0  Afghanistan      Asia  1952   28.801   8425333  779.445314       AFG   
1  Afghanistan      Asia  1957   30.332   9240934  820.853030       AFG   
2  Afghanistan      Asia  1962   31.997  10267083  853.100710       AFG   
3  Afghanistan      Asia  1967   34.020  11537966  836.197138       AFG   
4  Afghanistan      Asia  1972   36.088  13079460  739.981106       AFG   

   iso_num  
0        4  
1        4  
2        4  
3        4  
4        4  
